In [1]:
import sympy as sp
import numpy as np

In [317]:
x=sp.Symbol('x')
func1=sp.sqrt(0.4*x**2+1.5)/(2.5+sp.sqrt(2*x+0.8))
a1, b1 = 0.8, 2.4
func2=sp.cos(x**2+0.8)/(1.5+sp.sin(0.6*x+0.5))
a2, b2 = 0.4, 1.2

In [318]:
def int_rectangles(func, a, b, n, how='left'):
    h=(b-a)/n
    s=0
    if how=='left':
        for x_k in np.arange(a, b, h):
            s+=func.subs(x, x_k)
    elif how=='right':
        for x_k in np.arange(a+h, b+h, h):
            s+=func.subs(x, x_k)
    elif how=='mid':
        for x_k in np.arange(a+h, b+h, h):
            s+=func.subs(x, (x_k+x_k-h)/2)
    return sp.N(s*h, 10)

In [319]:
print(f'left, func1: {int_rectangles(func1, a1, b1, 10, "left")}')
print(f'right, func1: {int_rectangles(func1, a1, b1, 10, "right")}')
print(f'mid, func1: {int_rectangles(func1, a1, b1, 10, "mid")}')
print()
print(f'left, func2: {int_rectangles(func2, a2, b2, 10, "left")}')
print(f'right, func2: {int_rectangles(func2, a2, b2, 10, "right")}')
print(f'mid, func2: {int_rectangles(func2, a2, b2, 10, "mid")}')

left, func1: 0.5648046910
right, func1: 0.6424135173


mid, func1: 0.6354783346

left, func2: 0.05035536113
right, func2: -0.01595519438
mid, func2: 0.007264764935


In [337]:
def simpson(func, a, b, n):
    h=(b-a)/n
    s=0
    s+=(func.subs(x,a)+func.subs(x,b))/2
    for x_k in np.arange(a+h, b+h, h):
        s+=2*func.subs(x, (x_k+x_k-h)/2)
    for x_k in np.arange(a+h, b, h):
        s+=func.subs(x, x_k)
    return sp.N(s*h/3, 10)

In [338]:
print(f'Simpson, func1: {simpson(func1, a1, b1, 10)}')

Simpson, func1: 0.6352561666


In [339]:
print(sp.simplify(sp.diff((1-x**2)**6, x, 6)))
roots_leg=np.roots((665280, 0, -907200, 0, 302400, 0, -14400))
print(roots_leg)

665280*x**6 - 907200*x**4 + 302400*x**2 - 14400
[-0.93246951 -0.66120939  0.93246951  0.66120939 -0.23861919  0.23861919]


In [322]:
def transform(a, b, t):
    return (a+b)/2 +(b-a)/2*t

    
def int_gauss(func, a, b, roots_leg):
    A1, A2, A3, A4, A5, A6=sp.symbols('A1 A2 A3 A4 A5 A6')
    x1, x2, x3, x4, x5, x6 = roots_leg
    x1, x2, x3, x4, x5, x6 = transform(a, b, x1), transform(a, b, x2), transform(a, b, x3), transform(a, b, x4),\
                            transform(a, b, x5), transform(a, b, x6)
    eq1=A1 + A2 + A3 + A4 + A5 + A6 - sp.integrate(1, (x, a, b))
    eq2=A1*x1 + A2*x2 + A3*x3 + A4*x4 +\
        A5*x5 + A6*x6 - sp.integrate(x, (x, a, b))
    eq3=A1*x1**2 + A2*x2**2 + A3*x3**2 + A4*x4**2 +\
        A5*x5**2 + A6*x6**2 - sp.integrate(x**2, (x, a, b))
    eq4=A1*x1**3 + A2*x2**3 + A3*x3**3 + A4*x4**3 +\
        A5*x5**3 + A6*x6**3 - sp.integrate(x**3, (x, a, b))
    eq5=A1*x1**4 + A2*x2**4 + A3*x3**4 + A4*x4**4 +\
        A5*x5**4 + A6*x6**4 - sp.integrate(x**4, (x, a, b))
    eq6=A1*x1**5 + A2*x2**5 + A3*x3**5 + A4*x4**5 +\
        A5*x5**5 + A6*x6**5 - sp.integrate(x**5, (x, a, b))
    A1_, A2_, A3_, A4_, A5_, A6_ = list(sp.linsolve([eq1, eq2, eq3, eq4, eq5, eq6], A1, A2, A3, A4, A5, A6))[0]
    return sp.N((A1_*func.subs(x,x1)+A2_*func.subs(x,x2)+A3_*func.subs(x,x3)+A4_*func.subs(x,x4)+A5_*func.subs(x,x5)+A6_*func.subs(x,x6)), 10)
    

In [323]:
int_gauss(func2, a2, b2, roots_leg)

0.02984197538

In [324]:
def fund_polynom(a, b, n, k:int) -> sp.Function:
    h=(b-a)/n
    x_k=[a + j*h for j in range(n+1)]
    fund_poly = 1
    for j in range(len(x_k)):
        if j!=k:
            fund_poly*=(x-x_k[j])/(x_k[k]-x_k[j])
    return fund_poly

In [325]:
def interpol_int(func, a, b, n):
    h=(b-a)/n
    x_k=[a + k*h for k in range(n+1)]
    s=0
    for i in range(n+1):
       s+=sp.integrate(fund_polynom(a, b, n, i), (x, a, b))*func.subs(x, x_k[i])
    return sp.N(s, 10)


interpol_int(func2, a2, b2, 10)

0.02984196016

In [344]:
def int_gauss_cheb(func, a, b):
    nodes_cheb=[sp.cos((2*k-1)/12*sp.pi) for k in range(1, 7)]
    A1, A2, A3, A4, A5, A6=sp.symbols('A1 A2 A3 A4 A5 A6')
    p=1/sp.sqrt(1-x**2)
    x1, x2, x3, x4, x5, x6 = nodes_cheb
    x1, x2, x3, x4, x5, x6 = transform(a, b, x1), transform(a, b, x2), transform(a, b, x3), transform(a, b, x4),\
                            transform(a, b, x5), transform(a, b, x6)
    eq1=A1 + A2 + A3 + A4 + A5 + A6 - sp.integrate(p*1, (x, a, b))
    eq2=A1*x1 + A2*x2 + A3*x3 + A4*x4 +\
        A5*x5 + A6*x6 - sp.integrate(p*x, (x, a, b))
    eq3=A1*x1**2 + A2*x2**2 + A3*x3**2 + A4*x4**2 +\
        A5*x5**2 + A6*x6**2 - sp.integrate(p*x**2, (x, a, b))
    eq4=A1*x1**3 + A2*x2**3 + A3*x3**3 + A4*x4**3 +\
        A5*x5**3 + A6*x6**3 - sp.integrate(p*x**3, (x, a, b))
    eq5=A1*x1**4 + A2*x2**4 + A3*x3**4 + A4*x4**4 +\
        A5*x5**4 + A6*x6**4 - sp.integrate(p*x**4, (x, a, b))
    eq6=A1*x1**5 + A2*x2**5 + A3*x3**5 + A4*x4**5 +\
        A5*x5**5 + A6*x6**5 - sp.integrate(p*x**5, (x, a, b))
    A1_, A2_, A3_, A4_, A5_, A6_ = list(sp.linsolve([eq1, eq2, eq3, eq4, eq5, eq6], A1, A2, A3, A4, A5, A6))[0]
    return sp.N((A1_*func.subs(x,x1)+A2_*func.subs(x,x2)+A3_*func.subs(x,x3)+A4_*func.subs(x,x4)+A5_*func.subs(x,x5)+A6_*func.subs(x,x6)), 10)
    

In [345]:
int_gauss_cheb(func1, -1, 1)

1.330310672

### $\text{Для прямоугольников}: |R_n| <= \underset{x=\in [a,b]}{max}|f'(x)|*\frac{(b-a)^2}{n}, \quad f \in C^1[a,b]$
### $\text{Для средних прямоугольников}: |R_n| <= \underset{x=\in [a,b]}{max}|f''(x)|*\frac{(b-a)^3}{24n^2}, \quad f \in C^2[a,b]$
### $\text{Для формулы Симпсона}: |R_n| <= \underset{x=\in [a,b]}{max}|f'''(x)|*\frac{(b-a)^4}{192n^3}, \quad f \in C^3[a,b]$
### $\text{Для квадратурной формулы Гаусса}: |R_n| <= \frac{\underset{x=\in [a,b]}{max}|f^{(2n)}(x)|}{(2n)!}\int_a^b \rho(x)w_n^2(x) dx, \quad f \in C^{2n}[a,b]$
#### $\text{Для интерполяционной квадратурной формулы}: |R_n| <= \frac{\underset{x=\in [a,b]}{max}|f^{(n)}(x)|}{n!}\int_a^b \rho(x)|w_n(x)| dx, \quad f \in C^n[a,b]$


In [349]:
from scipy.optimize import minimize
from scipy.optimize import Bounds

def error_rate_rect(func, a, b, n):
    func_diff=func.diff()
    func_maximum=-minimize(sp.lambdify(x, -sp.Abs(func_diff)), a, bounds=Bounds(a,b))['fun']
    return func_maximum*(b-a)**2/n

def error_rate_rect_mid(func, a, b, n):
    func_diff_2=func.diff(x, 2)
    func_maximum=-minimize(sp.lambdify(x, -sp.Abs(func_diff_2)), a, bounds=Bounds(a,b))['fun']
    return func_maximum*(b-a)**3/(24*n**2)

def error_rate_simpson(func, a, b, n):
    func_diff_3=func.diff(x, 3)
    func_maximum=-minimize(sp.lambdify(x, -sp.Abs(func_diff_3)), a, bounds=Bounds(a,b))['fun']
    return np.abs(func_maximum)*(b-a)**4/(192*n**3)

def error_rate_gauss(func, a, b, n, is_p_cheb=False):
    func_diff_2n=func1.diff(x, 2*n)
    h=(b-a)/n
    x_k=[a + k*h for k in range(n+1)]
    w_n=1
    for x_j in x_k:
        w_n*=(x-x_j)
    # w_n=sp.Abs(w_n)
    func_maximum=-minimize(sp.lambdify(x, -sp.Abs(func_diff_2n)), a, bounds=Bounds(a,b))['fun']
    if is_p_cheb:
        return np.abs(func_maximum*sp.integrate(w_n**2*1/sp.sqrt(1-x**2), (x, a, b))/sp.factorial(2*n))
    else:
        return np.abs(func_maximum)*sp.integrate(w_n**2, (x, a, b))/sp.factorial(2*n)

def error_rate_interpol(func, a, b, n):
    func_diff_10=func1.diff(x, n)
    h=(b-a)/n
    x_k=[a + k*h for k in range(n+1)]
    w_n=1
    for x_j in x_k:
        w_n*=(x-x_j)
    func_maximum=-minimize(sp.lambdify(x, -sp.Abs(func_diff_10)), a, bounds=Bounds(a,b))['fun']
    return np.abs(func_maximum*sp.integrate(w_n, (x, a, b))/sp.factorial(n))

print(f'Оценка для формул прямоугольников для первой функции: {error_rate_rect(func1, a1, b1, 10)}')
print(f'Оценка для формул прямоугольников для второй функции: {error_rate_rect(func2, a2, b2, 10)}')
print(f'Оценка для формулы средних прямоугольников для первой функции: {error_rate_rect_mid(func1, a1, b1, 10)}')
print(f'Оценка для формулы средних прямоугольников для второй функции: {error_rate_rect_mid(func2, a2, b2, 10)}')
print(f'Оценка для формулы Симпсона: {error_rate_simpson(func1, a1, b1, 10)}')
print(f'Оценка для квадратурной формулы Гаусса: {error_rate_gauss(func2, a2, b2, 6)}')
print(f'Оценка для интерполяционной квадратурной формулы: {error_rate_interpol(func2, a2, b2, 10)}')
print(f'Оценка для интерполяционной квадратурной формулы с p=1/sqrt(1-x**2): {error_rate_gauss(func1, -1, 1, 6, True)}')


Оценка для формул прямоугольников для первой функции: 0.016983548994834192
Оценка для формул прямоугольников для второй функции: 0.05193494939706865
Оценка для формулы средних прямоугольников для первой функции: 0.00014171394954887103
Оценка для формулы средних прямоугольников для второй функции: 0.00018651332325143725
Оценка для формулы Симпсона: 3.420428415141387e-06
Оценка для квадратурной формулы Гаусса: 1.72770311347738E-11
Оценка для интерполяционной квадратурной формулы: 1.24344168033832E-15
Оценка для интерполяционной квадратурной формулы с p=1/sqrt(1-x**2): 4.68035303824930E-11
